In [2]:
class DoublyLinkedBase:
    class Node:
        __slots__ = 'element' , 'prev' , 'next'
        def __init__(self, element, prev, next): # initialize node’s fields
            self.element = element # user’s element
            self.prev = prev # previous node reference
            self.next = next
    def __init__(self):
        self.header = self.Node(None, None, None)
        self.trailer = self.Node(None, None, None)
        self.header.next = self.trailer # trailer is after header
        self.trailer.prev = self.header # header is before trailer
        self.size = 0 # number of elements
    def __len__(self):
        return self.size
    def is_empty(self):
        return self.size == 0
    def insert_between(self, e, predecessor, successor):
        newest = self.Node(e, predecessor, successor) # linked to neighbors
        predecessor.next = newest
        successor.prev = newest
        self.size+=1
        return newest
    def delete_node(self,node):
        predecessor = node.prev
        successor = node.next
        predecessor.next = successor
        successor.prev = predecessor
        self.size=self.size-1
        element = node.element # record deleted element
        node.prev = node.next = node.element = None # deprecate node
        return element # return deleted element
class PositionalList(DoublyLinkedBase):
    class Position:
        def __init__(self, container, node):
            self.container = container
            self.node = node
        def element(self):
            return self.node.element
        def eq (self, other):
            return type(other) is type(self) and other. node is self. node
        def ne (self, other):
            return not(self == other) # opposite of eq
    def validate(self, p):
        if not isinstance(p, self.Position):
            raise TypeError("p must be proper Position type")
        if p. container is not self:
            raise ValueError("p does not belong to this container")
        if p. node. next is None: # convention for deprecated nodes
            raise ValueError("p is no longer valid")
        return p. node
    def make_position(self, node):
        if node is self.header or node is self.trailer:
            return None
        else:
            return self.Position(self, node) # legitimate position
    def first(self):
        return self.make_position(self.header.next)
    def last(self):
        return self.make_position(self.trailer.prev)
    def before(self, p):
        node = self.validate(p)
        return self.make_position(node.prev)
    def after(self, p):
        node = self.validate(p)
        return self.make_position(node.next)
    def __iter__(self):
        cursor = self.first()
        while cursor is not None:
            yield cursor.element()
            cursor = self.after(cursor)
    def insert_between(self, e, predecessor, successor):
        node = super().insert_between(e, predecessor, successor)
        return self.make_position(node)
    def add_first(self, e):
        return self.insert_between(e, self.header, self.header.next)
    def add_last(self, e):
        return self.insert_between(e, self.trailer.prev, self.trailer)
    def add_before(self, p, e):
        original = self.validate(p)
        return self.insert_between(e, original.prev, original)
    def add_after(self, p, e):
        original = self.validate(p)
        return self.insert_between(e, original, original.next)
    def delete(self, p):
        original = self.validate(p)
        return self.delete_node(original) # inherited method returns element
    def replace(self, p, e):
        original = self.validate(p)
        old_value = original.element # temporarily store old element
        original.element = e # replace with new element
        return old_value
F=PositionalList()
t=F.add_first(4)
h=F.add_after(t,6)

In [5]:
import math
print(math.ceil(math.sqrt(4)))

2


In [3]:
class PriorityQueueBase:
    class Item:
        __slots__='key','value'
        def __init__(self,k,v):
            self.key=k
            self.value=v
        def __lt__(self,other):
            return self.key<other.key
    def is_empty(self):
        return len(self)==0

In [41]:
'''This is an Unordered PositionalList implementation of Heap
'Unordered' refers to the fact that we insert the data first and then,
we search for the minimum element in the PositionalList.
This makes add operation in O(1) time while
remove_min or min(minsky) operation takes O(n) time.
'''

class UnorderedPQ(PriorityQueueBase):
    def find_min(self):
        if(self.is_empty()):
            raise Empty("empty")
        small=self.data.first()
        walk=self.data.after(small)
        while(walk is not None):
            if(walk.element() < small.element()):
                small=walk
            walk=self.data.after(walk)
        return small
    def __init__(self):
        self.data=PositionalList()
    def __len__(self):
        return len(self.data)
    def add(self,key,value):
        self.data.add_last(self.Item(key,value))
    def minsky(self):
        p=self.find_min()
        item=p.element()
        return (item.key,item.value)
    def remove_min(self):
        p=self.find_min()
        item=self.data.delete(p)
        return (item.key,item.value)
L=UnorderedPQ()
L.add(4,'B')
L.add(6,'C')
L.add(1,'F')
L.add(1,'A')
print(L.minsky())        
print(L.remove_min())
print(L.minsky())

(1, 'F')
(1, 'F')
(1, 'A')


In [52]:
class OrderedPQ(PriorityQueueBase):
    def add(self,key,value):
        newest=self.Item(key,value)
        walk=self.data.last()
        while(walk is not None and newest<walk.element()):
            walk=self.data.before(walk)
        if(walk is None):
            self.data.add_first(newest)
        else:
            self.data.add_after(walk,newest)
    def minsky(self):
        f=self.data.first()
        f=f.element()
        return (f.key,f.value)
    def __init__(self):
        self.data=PositionalList()
    def __len__(self):
        return len(self.data)
    def remove_min(self):
        p=self.data.delete(self.data.first())
        return (p.key,p.value)
PQ=OrderedPQ()
h=PQ.add(6,'A')
j=PQ.add(4,1)#ducktyping in practice
print(PQ.remove_min())
print(PQ.minsky())

(4, 1)
(6, 'A')


2 1


1 2 3 4


In [45]:
class PrioritQueueBase:
    class Item:
        __slots__='key','value'
        def __init__(self,k,v):
            self.key=k
            self.value=v
        def __lt__(self,other):
            return self.key < other.key
    def isempty(self):
        return len(self)==0


In [62]:
class HeapPriorityQueue(PriorityQueueBase):
    def is_empty(self):
        return len(self.data)==0
    def parent(self,j):
        return (j-1)//2
    def right(self,j):
        return (2*j+1)//2
    def left(self,j):
        return (2*j+2)//2
    def has_right(self,j):
        return self.right(j) < len(self.data)
    def has_left(self,j):
        return self.left(j) < len(self.data)
    def swap(self,i,j):
        self.data[i],self.data[j] = self.data[j],self.data[i]
    def upheap(self,j):
        parent=self.parent(j)
        #print(p.key)
        if(j>0 and self.data[j]<self.data[parent]):
            self.swap(j,parent)
            self.upheap(parent)
    def downheap(self,j):
        if(self.has_left(j)):
            #for i in range(len(self.data)):
                #print(self.data[i].key,self.data[i].value,i)
            left=self.left(j)
            small_child=left
            if(self.has_right(j)):
                right=self.right(j)
                if(self.data[right]<self.data[left]):
                    small_child=right
                if(self.data[small_child]<self.data[j]):
                    self.swap(j,small_child)
                    self.downheap(small_child)
    def __init__(self,contents=()):
        self.data=[self.Item(k,v) for k,v in contents]
        if(len(self.data)>1):
            print(self.data)
            self.heapify()
    def heapify(self):
        start=self.parent(len(self.data)-1)
        for j in range(start,0,-1):
            print(self.data[j].key)
            self.downheap(j)
    def minsky(self):
        if(self.is_empty()):
            raise Empty('empty')
        item=self.data[0]
        return item.key,item.value
    def remove_min(self):
        if(self.is_empty()):
            raise Empty('empty')
        self.swap(0,len(self.data)-1)
        item=self.data.pop()
        self.downheap(0)
        return item.key,item.value
    def add(self,i,k):
        t=self.Item(i,k)
        self.data.append(t)
        self.upheap(len(self.data)-1)
t=HeapPriorityQueue(((6,'l'),(9,'sd'),(0.1,'k')))
t.add(5,'A')
t.add(4,'A')
t.add(6,'A')
t.add(1,'A')
print(t.minsky())
print(t.remove_min())
print(t.minsky())

[<__main__.PriorityQueueBase.Item object at 0x000001EDA0D33908>, <__main__.PriorityQueueBase.Item object at 0x000001EDA0D33788>, <__main__.PriorityQueueBase.Item object at 0x000001EDA0D33748>]
(4, 'A')
(4, 'A')
(1, 'A')


In [1]:
'''Pythons Heapq module'''
import heapq
L=[1,6,5,3,2,7]
heapq.heapify(L)
print(L)
heapq.heappush(L,4)
print(heapq.heappop(L))
heapq.heappushpop(L,5)
print(L)
heapq.heapreplace(L,10)
print(L)
print(heapq.nlargest(4,L))
print(heapq.nsmallest(4,L))

[1, 2, 5, 3, 6, 7]
1
[3, 5, 4, 5, 6, 7]
[4, 5, 7, 5, 6, 10]
[10, 7, 6, 5]
[4, 5, 5, 6]


In [ ]:
def ag(a=0):
    a+=1
    ag(a)
    print(a)
t=0
ag()
print(t)